In [1]:
from tensorflow import keras

keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")


C:\Users\keonhi\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\keonhi\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\keonhi\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\keonhi\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

In [3]:
#He initialization is by default fan_in.
#If you want to use fan_avg you should use VarianceScaling()

he_avg=keras.initializers.VarianceScaling(scale=2, mode='fan_avg', distribution="uniform")
keras.layers.Dense(10, activation="relu", kernel_initializer=he_avg)

In [4]:
he_avg=keras.initializers.VarianceScaling(scale=2, mode='fan_avg', distribution="normal")
keras.layers.Dense(10, activation="relu", kernel_initializer=he_avg)

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


In [7]:
model= keras.models.Sequential([
    keras.layers.Input(10),
    keras.layers.Dense(30, activation="leaky_relu", kernel_initializer="he_normal")
])

ValueError: Unknown activation function:leaky_relu

In [9]:
model= keras.models.Sequential([
    keras.layers.Input(10),
    keras.layers.Dense(30, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2)
])

In [12]:
model2=keras.models.Sequential([
    keras.layers.Dense(30, kernel_initializer="he_normal"),
    keras.layers.PReLU()
])

In [14]:
layer= keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal")

In [19]:
model3 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [21]:
model3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 784)               3136      
_________________________________________________________________
dense_12 (Dense)             (None, 300)               235500    
_________________________________________________________________
batch_normalization_2 (Batch (None, 300)               1200      
_________________________________________________________________
dense_13 (Dense)             (None, 100)               30100     
_________________________________________________________________
batch_normalization_3 (Batch (None, 100)               400       
_________________________________________________________________
dense_14 (Dense)             (None, 10)               

In [22]:
[(var.name, var.trainable) for var in model3.layers[1].variables]

[('batch_normalization_1/gamma:0', True),
 ('batch_normalization_1/beta:0', True),
 ('batch_normalization_1/moving_mean:0', False),
 ('batch_normalization_1/moving_variance:0', False)]

In [24]:
model3.layers[1].updates

[<tf.Operation 'batch_normalization_1/cond_2/Merge' type=Merge>,
 <tf.Operation 'batch_normalization_1/cond_3/Merge' type=Merge>]

In [25]:
model3 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [32]:
modelA=keras.models.load_model("keras_model.h5")
modelB=keras.models.Sequential(modelA.layers[:-3])
modelB.add(keras.layers.Dense(30, activation="sigmoid"))

In [36]:
modelA_clone=keras.models.clone_model(modelA)
modelA_clone.set_weights(modelA.get_weights())

In [37]:
#freeze reused layers
for layer in modelB.layers[:-1]:
    layer.trainable =False

In [38]:
modelB.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [39]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing=fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_valid=scaler.transform(X_valid)
X_test=scaler.transform(X_test)


In [40]:
history=modelB.fit(X_train, y_train, epochs=4, validation_data=(X_valid, y_valid))

for layer in modelB.layers[:-1]:
    layer.trainable=True
    
#after unfreezing the reused layers, it is better to lower the learning rate.
optimizer=keras.optimizers.SGD(lr=1e-4)
modelB.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history=modelB.fit(X_train, y_train, epochs=16, validation_data=(X_valid, y_valid))

ValueError: Error when checking input: expected deep_input to have shape (6,) but got array with shape (8,)

In [42]:
optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.9)

In [43]:
optimizer=keras.optimizers.RMSprop(lr=0.001, rho=0.9)

In [44]:
optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)